In [23]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
from google.cloud import storage
import numpy as np 
from shapely.geometry import Point
import json


# ee.Authenticate()
ee.Initialize()


Constants

In [36]:
s2_str_path='COPERNICUS/S2_SR_HARMONIZED'
CLOUDY_PIXEL_PERCENTAGE_STR ='CLOUDY_PIXEL_PERCENTAGE'
CLOUDY_PIXEL_PERCENTAGE_INT =20

In [31]:
# Load geopackage / shapefile 

def load_geometry(
    path : str , #path to geometry , can be .shp or .gpkg, didn't tested other geometries
    crs : str , #the CRS of the given geoemtry
    ):

    
    gdf= gpd.read_file(path)
    gdf = gdf.set_crs(crs)

    return gdf

    

Example

In [32]:
gdf=load_geometry(path=r"geometry\test_bbox.gpkg",
                  crs='4326')
gdf

,id,name,geometry
0,1,1,"MULTIPOLYGON (((34.19874 31.59613, 34.63046 31..."


Geometry to ee.featurecollection

In [33]:
def gdf_to_fc(
  gdf : gpd.GeoDataFrame , #
  ):
  
  geo_json = gdf.to_json()

  featureCollection = ee.FeatureCollection(json.loads(geo_json))

  return featureCollection
  

In [34]:
fct=gdf_to_fc(gdf = gdf)

Access S2

In [41]:
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

In [42]:

def access_s2(
        geometry : ee.FeatureCollection , # Geometry in type of featurecollection
        start_date : str ,#date  in the form of 'yyyy-mm-dd'
        end_date : str , #daate in the form of 'yyyy-mm-dd'
        s2_path : str = s2_str_path,
        ):
        image_collection= ee.ImageCollection(s2_path).filter(ee.Filter.lt(CLOUDY_PIXEL_PERCENTAGE_STR,CLOUDY_PIXEL_PERCENTAGE_INT)).filterBounds(geometry).map(maskS2clouds).map(lambda img: img.clip(geometry))



        return image_collection

In [40]:
img=access_s2(
        geometry = fct,
        start_date = '2023-10-06',
        end_date ='2023-10-08')